In [31]:
!date

Tue Mar 28 10:01:38 CST 2017


In [1]:
import pandas as pd
import os
import re
import shutil

In [2]:
path = './source'

In [3]:
indicators = set()

for f in os.listdir(path):
    if re.match('ddf--datapoints--[a-z0-9_]*--.*', f):
        g = re.match('ddf--datapoints--([a-z0-9_]*)--.*', f).groups()[0]
        if g.endswith('_male') or g.endswith('_female'):
            continue
        indicators.add(g)
        
for i in indicators:
    print(i)

immigration_surplus_employed_aged_20_64
population_20xx_12_31
indicators
mean_income_aged_lt_20
immigration_surplus_min_3_years_of_higher_education_aged_25_64
higher_education_min_3_years_percent_of_population_aged_25_64
immigration_surplus
cumulative_immigration_surplus
employment_rate_aged_20_64
cumulative_immigration_surplus_employed_aged_20_64
cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64


In [26]:
for indicator in indicators:
    
    if indicator == 'indicators':
        continue
    
    ilist = [
        indicator,
        indicator+'_male',
        indicator+'_female'
    ]
    
    data = []

    for i in ilist:
        d = pd.read_csv(os.path.join(path, 
                                     'ddf--datapoints--{}--by--basomrade--year.csv'.format(i)))
        d = d.rename(columns={i:indicator})
        if 'female' in i:
            d['gender'] = 'female'
        elif 'male' in i:
            d['gender'] = 'male'
        else:
            shutil.copy(os.path.join(path, 
                                     'ddf--datapoints--{}--by--basomrade--year.csv'.format(i)),
                        'out/')
            continue
        data.append(d)  
        
    finaldata = pd.concat(data, ignore_index=True)
    finaldata.to_csv(
        'out/ddf--datapoints--{}--by--gender--basomrade--year.csv'.format(indicator),
        index=False
    )

In [21]:
!open .

In [9]:
indicator_by_mun = '../ddf--datapoints--indicators--by--municipality--year.csv'

In [10]:
dm = pd.read_csv(indicator_by_mun)

In [11]:
dm = dm.set_index(['municipality', 'year'])

In [12]:
dm_data = {}

for name, ser in dm.iteritems():
    if name.endswith('_male'):
        gend = 'male'
        d = ser.copy()
        d.name = name.replace('_male', '')
    elif name.endswith('_female'):
        gend = 'female'
        d = ser.copy()
        d.name = name.replace('_female', '')
    else:
        continue
        gend = 'both'
        d = ser.copy()
    
    n = d.name
    d = d.reset_index()
    d['gender'] = gend
    
    if n in dm_data.keys():
        dm_data[n].append(d.set_index(['gender', 'municipality', 'year']))
    else:
        dm_data[n] = [d.set_index(['gender', 'municipality', 'year'])]

In [13]:
dm_data.keys()

dict_keys(['median_income_aged_lt_20', 'mean_income_aged_lt_20', 'population_20xx_12_31', 'higher_education_min_3_years_percent_of_population_aged_25_64', 'employment_rate_aged_20_64', 'immigration_surplus_employed_aged_20_64', 'cumulative_immigration_surplus_employed_aged_20_64', 'immigration_surplus_min_3_years_of_higher_education_aged_25_64', 'cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64', 'immigration_surplus', 'cumulative_immigration_surplus'])

In [14]:
dm_comp_0 = [pd.concat(x) for x in dm_data.values()]

In [15]:
dm_comp = pd.concat(dm_comp_0, axis=1)

In [16]:
dm_comp.query('municipality == "0114_upplands_vasby"')

median_income_aged_lt_20  \
gender municipality        year                             
male   0114_upplands_vasby 2000                    228025   
                           2001                    237543   
                           2002                    244254   
                           2003                    249028   
                           2004                    252281   
                           2005                    255941   
                           2006                    262854   
                           2007                    273721   
                           2008                    284700   
                           2009                    286293   
                           2010                    287952   
                           2011                    295944   
                           2012                    302632   
                           2013                    306782   
                           2014                    311556   
female 0114_upplands_vasby 2000                    174591   
                           2001                    182445   
                           2002                    188775   
                           2003                    194136   
                           2004                    196212   
                           2005                    198749   
                           2006                    203756   
                           2007                    211081   
                           2008                    219154   
                           2009                    223094   
                           2010                    222028   
                           2011                    225000   
                           2012                    231977   
                           2013                    236131   
                           2014                    239959   

                                 mean_income_aged_lt_20  \
gender municipality        year                           
male   0114_upplands_vasby 2000                  253133   
                           2001                  263753   
                           2002                  269020   
                           2003                  272043   
                           2004                  274568   
                           2005                  281940   
                           2006                  288900   
                           2007                  299825   
                           2008                  312189   
                           2009                  310700   
                           2010                  312978   
                           2011                  321287   
                           2012                  328922   
                           2013                  331518   
                           2014                  337404   
female 0114_upplands_vasby 2000                  175400   
                           2001                  185234   
                           2002                  192332   
                           2003                  199144   
                           2004                  202604   
                           2005                  206920   
                           2006                  211908   
                           2007                  219537   
                           2008                  228675   
                           2009                  234378   
                           2010                  235549   
                           2011                  240000   
                           2012                  246665   
                           2013                  252084   
                           2014                  257900   

                                 population_20xx_12_31  \
gender municipality        year                          
male   0114_upplands_vasby 2000                  18658   
                 

In [17]:
dm_comp.to_csv('out/ddf--datapoints--indicators--by--gender--municipality--year.csv')

In [18]:
dm[dm_comp.columns].to_csv('out/ddf--datapoints--indicators--by--municipality--year.csv')

In [145]:
set(dm.columns)

{'cumulative_immigration_surplus',
 'cumulative_immigration_surplus_employed_aged_20_64',
 'cumulative_immigration_surplus_employed_aged_20_64_female',
 'cumulative_immigration_surplus_employed_aged_20_64_male',
 'cumulative_immigration_surplus_female',
 'cumulative_immigration_surplus_male',
 'cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64',
 'cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64_female',
 'cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64_male',
 'employment_rate_aged_20_64',
 'employment_rate_aged_20_64_female',
 'employment_rate_aged_20_64_male',
 'higher_education_min_3_years_percent_of_population_aged_25_64',
 'higher_education_min_3_years_percent_of_population_aged_25_64_female',
 'higher_education_min_3_years_percent_of_population_aged_25_64_male',
 'immigration_surplus',
 'immigration_surplus_employed_aged_20_64',
 'immigration_surplus_employed_aged_20_64_female',
 'immigration_surplus_empl

In [19]:
dm.query('municipality == "0114_upplands_vasby"')

population_20xx_12_31  \
municipality        year                          
0114_upplands_vasby 2000                  37576   
                    2001                  37524   
                    2002                  37444   
                    2003                  37397   
                    2004                  37517   
                    2005                  37624   
                    2006                  37848   
                    2007                  38055   
                    2008                  38248   
                    2009                  38641   
                    2010                  39289   
                    2011                  40194   
                    2012                  40723   
                    2013                  41449   
                    2014                  41816   

                          median_income_aged_lt_20_male  \
municipality        year                                  
0114_upplands_vasby 2000                         228025   
                    2001                         237543   
                    2002                         244254   
                    2003                         249028   
                    2004                         252281   
                    2005                         255941   
                    2006                         262854   
                    2007                         273721   
                    2008                         284700   
                    2009                         286293   
                    2010                         287952   
                    2011                         295944   
                    2012                         302632   
                    2013                         306782   
                    2014                         311556   

                          median_income_aged_lt_20_female  \
municipality        year                                    
0114_upplands_vasby 2000                           174591   
                    2001                           182445   
                    2002                           188775   
                    2003                           194136   
                    2004                           196212   
                    2005                           198749   
                    2006                           203756   
                    2007                           211081   
                    2008                           219154   
                    2009                           223094   
                    2010                           222028   
                    2011                           225000   
                    2012                           231977   
                    2013                           236131   
                    2014                           239959   

                          median_income_aged_lt_20  \
municipality        year                             
0114_upplands_vasby 2000                    196879   
                    2001                    205625   
                    2002                    212482   
                    2003                    217412   
                    2004                    219652   
                    2005                    222945   
                    2006                    229105   
                    2007                    237901   
                    2008                    247208   
                    2009                    250920   
                    2010                    251579   
                    2011                    257418   
                    2012                    263614   
                    2013                    267678   
                    2014                    272692   

                          mean_income_aged_lt_20_male  \
municipality        year                                
0114_upplands_vasby 2000                       253133   
                    2001  

In [30]:
import numpy as np

for indicator in indicators:
    
    if indicator == 'indicators':
        continue
    
    ilist = [
        indicator,
        indicator+'_male',
        indicator+'_female'
    ]
    
    data = []

    for i in ilist:
        d = pd.read_csv(os.path.join(path, 
                                     'ddf--datapoints--{}--by--basomrade--year.csv'.format(i)))
        d = d.rename(columns={i:indicator})
        if 'female' in i:
            d['gender'] = 'female'
        elif 'male' in i:
            d['gender'] = 'male'
        else:
            d['gender'] = 'both'
        data.append(d)  
        
    finaldata = pd.concat(data, ignore_index=True)
    finaldata = finaldata.set_index(['basomrade', 'gender', 'year'])
    assert not np.all(finaldata.index.duplicated())
    print(indicator)
    print(finaldata.dropna().shape[0])

immigration_surplus_employed_aged_20_64
49161
population_20xx_12_31
49161
mean_income_aged_lt_20
49161
immigration_surplus_min_3_years_of_higher_education_aged_25_64
49161
higher_education_min_3_years_percent_of_population_aged_25_64
49153
immigration_surplus
49161
cumulative_immigration_surplus
49161
employment_rate_aged_20_64
49156
cumulative_immigration_surplus_employed_aged_20_64
49161
cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64
49161


In [24]:
for indicator in indicators:
    
    if indicator == 'indicators':
        continue
        
    ilist = [
        indicator,
        indicator+'_male',
        indicator+'_female'
    ]
    
    data = []
    count = 0

    for i in ilist:
        d = pd.read_csv(os.path.join(path, 
                                     'ddf--datapoints--{}--by--basomrade--year.csv'.format(i)))
#         d = d.rename(columns={i:indicator})
#         if 'male' in i:
#             d['gender'] = 'male'
#         elif 'female' in i:
#             d['gender'] = 'female'
#         else:
#             d['gender'] = 'both'
#         data.append(d)  
        count = count + d.shape[0]
        
#     finaldata = pd.concat(data, ignore_index=True)
    print(indicator)
    print(count)

immigration_surplus_employed_aged_20_64
49161
population_20xx_12_31
49161
mean_income_aged_lt_20
49161
immigration_surplus_min_3_years_of_higher_education_aged_25_64
49161
higher_education_min_3_years_percent_of_population_aged_25_64
49153
immigration_surplus
49161
cumulative_immigration_surplus
49161
employment_rate_aged_20_64
49156
cumulative_immigration_surplus_employed_aged_20_64
49161
cumulative_immigration_surplus_min_3_years_of_higher_education_aged_25_64
49161
